In [197]:
%%capture
%pip install pandas
%pip install torch
%pip install scikit-learn

In [555]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [556]:
class Model(nn.Module):
    # in_features is 17 (9 original + 4 mean + 4 sd features)
    def __init__(self, in_features=17, h1=60, h2=30): 
        super().__init__()
        # Layer 1
        self.fc1 = nn.Linear(in_features, h1)
        self.drop1 = nn.Dropout(p=0.3)
        # Layer 2
        self.fc2 = nn.Linear(h1, h2)
        self.drop2 = nn.Dropout(p=0.3)
        # Final Layer: 1 output neuron
        self.out = nn.Linear(h2, 1) 

    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = self.drop1(x)
        x = f.relu(self.fc2(x))
        x = self.drop2(x)
        return self.out(x)

In [557]:
torch.manual_seed(67)

dataset_path = "./"

automated_dataset = pd.read_json(dataset_path + "automatedAccountData.json")
non_automated_dataset = pd.read_json(dataset_path + "nonautomatedAccountData.json")

In [558]:
combined = pd.concat([automated_dataset, non_automated_dataset], axis=0)

combined

,userMediaCount,mediaLikeNumbers,mediaCommentNumbers,mediaCommentsAreDisabled,mediaHashtagNumbers,mediaUploadTimes,mediaHasLocationInfo,userFollowerCount,userFollowingCount,userHasHighlighReels,userHasExternalUrl,userTagsCount,userBiographyLength,usernameLength,usernameDigitCount,automatedBehaviour
0,97,"[100, 78, 112, 78, 77, 111, 97, 95, 63, 83, 42...","[5, 2, 10, 0, 3, 2, 2, 2, 4, 0, 0, 6, 6, 0, 2,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 10, 12, 10, 13, 13, 10, 12, 10, 11, 11, 1...","[1540847457, 1540744841, 1538410400, 153729214...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1612,7108,0,1,36,133,12,0,1
1,863,"[229, 331, 180, 326, 313, 157, 245, 161, 182, ...","[3, 2, 3, 4, 7, 2, 0, 6, 2, 2, 1, 9, 2, 0, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 14, 17, 12, 10, 13, 14, 14, 11, 10, 11...","[1542664246, 1542569444, 1542564418, 154211928...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3028,912,1,0,77,99,11,0,1
2,149,"[237, 254, 350, 257, 180, 172, 141, 84, 218, 8...","[12, 16, 37, 11, 3, 4, 2, 4, 5, 1, 1, 2, 3, 3,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25, 25, 25, 25, 15, 15, 15, 5, 15, 10, 10, 10...","[1543220394, 1542972220, 1542868998, 154278889...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",5213,1157,1,1,56,120,18,0,1
3,33,"[163, 30, 92, 111, 145, 110, 81, 81, 61, 60, 6...","[2, 2, 7, 14, 17, 3, 2, 5, 2, 1, 5, 5, 8, 1, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 14, 14, 0, 0...","[1543886933, 1543793413, 1543704343, 154344770...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",12423,1763,0,0,2,55,13,0,1
4,1000,"[61, 222, 437, 829, 681, 440, 920, 256, 1017, ...","[4, 0, 13, 9, 9, 6, 16, 4, 25, 6, 3, 4, 16, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 1, 2, 3, 3, 0, 0, 1, 0, 2, 1, 2, 3, 1, 1, ...","[1543939920, 1543699089, 1543187756, 154264414...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",9496,2412,1,0,91,33,8,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,26,"[117, 82, 70, 50, 94, 45, 40, 47, 44, 43, 45, ...","[0, 1, 0, 0, 4, 0, 0, 2, 1, 0, 5, 6, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 0, 2, ...","[1533665377, 1529705967, 1525534881, 145340569...","[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, ...",303,335,0,0,23,22,12,0,0
696,12,"[86, 87, 75, 24, 30, 29, 22, 33, 34, 29, 17, 13]","[4, 5, 10, 0, 0, 1, 1, 2, 0, 0, 3, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1541858886, 1539619797, 1537199999, 146400347...","[1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]",270,318,0,0,10,23,11,0,0
697,11,"[28, 18, 20, 15, 8, 13, 17, 15, 10, 4, 11]","[0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[2, 10, 6, 0, 3, 2, 6, 4, 0, 2, 0]","[1417601401, 1416411495, 1412454197, 140580953...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",175,179,0,0,4,6,5,0,0
698,116,"[146, 132, 123, 148, 122, 118, 130, 144, 119, ...","[10, 0, 0, 0, 2, 2, 4, 9, 4, 0, 4, 3, 3, 3, 3,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1540144489, 1534530033, 1532627229, 153228440...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, ...",311,311,0,0,13,0,10,0,0


In [559]:
# Columns that contain lists/arrays
sequence_cols = [
    "mediaLikeNumbers",
    "mediaCommentNumbers",
    "mediaHashtagNumbers",
    "mediaHasLocationInfo"
]

# Calculate Mean and Standard Deviation for all sequence columns
for col in sequence_cols:
    
    # 1. Calculate Standard Deviation (sd_) from the original list
    new_col_name = 'sd_' + col
    # We use a lambda to calculate SD only if the list is not empty, otherwise return 0.0
    combined[new_col_name] = combined[col].apply(lambda x: np.std(x) if len(x) > 0 else 0.0)

    # 2. Calculate Mean (replaces the original list column)
    # We use a lambda to calculate Mean only if the list is not empty, otherwise return 0.0
    combined[col] = combined[col].apply(lambda x: np.mean(x) if len(x) > 0 else 0.0)

# Apply the mean function to each list in the sequence columns
for col in sequence_cols:
    # Use .apply(np.mean) to calculate the mean of the list in each cell
    # Note: np.mean works on lists/arrays and handles empty lists gracefully (returns NaN)
    combined[col] = combined[col].apply(np.mean)

# Fill any NaN values (from empty lists) with 0.0 or a suitable default
combined = combined.fillna(0.0)

combined

,userMediaCount,mediaLikeNumbers,mediaCommentNumbers,mediaCommentsAreDisabled,mediaHashtagNumbers,mediaUploadTimes,mediaHasLocationInfo,userFollowerCount,userFollowingCount,userHasHighlighReels,userHasExternalUrl,userTagsCount,userBiographyLength,usernameLength,usernameDigitCount,automatedBehaviour,sd_mediaLikeNumbers,sd_mediaCommentNumbers,sd_mediaHashtagNumbers,sd_mediaHasLocationInfo
0,97,55.041237,2.453608,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",13.886598,"[1540847457, 1540744841, 1538410400, 153729214...",0.979381,1612,7108,0,1,36,133,12,0,1,29.795495,3.458206,5.759575,0.142104
1,863,61.212051,1.701043,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.115875,"[1542664246, 1542569444, 1542564418, 154211928...",0.505214,3028,912,1,0,77,99,11,0,1,110.261105,2.774422,3.200596,0.499973
2,149,70.939597,2.026846,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",10.302013,"[1543220394, 1542972220, 1542868998, 154278889...",0.248322,5213,1157,1,1,56,120,18,0,1,49.064022,3.803824,5.124890,0.432040
3,33,65.181818,4.787879,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.121212,"[1543886933, 1543793413, 1543704343, 154344770...",0.000000,12423,1763,0,0,2,55,13,0,1,34.639664,5.425850,3.967730,0.000000
4,1000,160.267000,3.585000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.008000,"[1543939920, 1543699089, 1543187756, 154264414...",0.269000,9496,2412,1,0,91,33,8,0,1,152.971042,4.013325,5.482877,0.443440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,26,39.346154,1.115385,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.500000,"[1533665377, 1529705967, 1525534881, 145340569...",0.307692,303,335,0,0,23,22,12,0,0,25.756168,1.804416,1.886185,0.461538
696,12,39.916667,2.166667,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.083333,"[1541858886, 1539619797, 1537199999, 146400347...",0.416667,270,318,0,0,10,23,11,0,0,25.519464,2.881936,0.276385,0.493007
697,11,14.454545,0.363636,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",3.181818,"[1417601401, 1416411495, 1412454197, 140580953...",0.000000,175,179,0,0,4,6,5,0,0,6.169774,0.771389,2.979267,0.000000
698,116,90.646552,3.836207,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.215517,"[1540144489, 1534530033, 1532627229, 153228440...",0.387931,311,311,0,0,13,0,10,0,0,33.299129,3.151201,0.469887,0.487279


In [560]:
X = combined.drop(["automatedBehaviour", "mediaUploadTimes", "mediaCommentsAreDisabled"], axis=1)
y = combined["automatedBehaviour"]
X = X.values
y = y.values

In [561]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=67)

In [562]:
from sklearn.preprocessing import StandardScaler

# 1. Initialize the Scaler
scaler = StandardScaler()

# 2. Fit the scaler ONLY on the training data to prevent data leakage
# X_train is still a NumPy array at this point
X_train = scaler.fit_transform(X_train)

# 3. Apply the transformation to the test data
X_test = scaler.transform(X_test)

In [563]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [564]:
model = Model()
criterion = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [565]:
epochs = 500
losses = []
for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss.detach().numpy())

    if i % 10 == 0:
        print(f"{i}) Loss = {loss}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0) Loss = 0.7125270366668701
10) Loss = 0.3128153681755066
20) Loss = 0.24740009009838104
30) Loss = 0.20847345888614655
40) Loss = 0.18829914927482605
50) Loss = 0.17542307078838348
60) Loss = 0.16444624960422516
70) Loss = 0.1518295556306839
80) Loss = 0.14932547509670258
90) Loss = 0.1456305980682373
100) Loss = 0.1412566900253296
110) Loss = 0.1455131620168686
120) Loss = 0.13445770740509033
130) Loss = 0.12461219727993011
140) Loss = 0.12866535782814026
150) Loss = 0.12448474019765854
160) Loss = 0.128595232963562
170) Loss = 0.12006432563066483
180) Loss = 0.11702525615692139
190) Loss = 0.10968749225139618
200) Loss = 0.10373038053512573
210) Loss = 0.10677739232778549
220) Loss = 0.10193696618080139
230) Loss = 0.10265596210956573
240) Loss = 0.09896652400493622
250) Loss = 0.10330424457788467
260) Loss = 0.09630611538887024
270) Loss = 0.09191994369029999
280) Loss = 0.0954609364271164
290) Loss = 0.08479391783475876
300) Loss = 0.09825869649648666
310) Loss = 0.09149608016014